In [1]:
import urllib
import requests
import pandas as pd
import re
import time
import pickle
from xml.etree import ElementTree
import xmltodict
import numpy as np
import os
import glob

In [ ]:
#fetch pmids read from a text file of mesh and search terms
import urllib.request
import requests
import xmltodict
import urllib.parse



# Function to read search terms from a text file
def read_search_terms_from_file(file_path):
    with open(file_path, 'r') as file:
        terms = file.read().splitlines()
    return terms

# Define the path to your text file
file_path = 'mesh_terms_refined.txt'

# Read search terms from the file
query_list = read_search_terms_from_file(file_path)

# Additional settings
base_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
db = 'db=pubmed'
mindate = "2021"
maxdate = "2024"
final_ids = []
query_ids = []
all_pmids = []
file_index = 0

for item in query_list:
    query = item
    print("searching query about:", query)

    # esearch specific settings
    search_eutil = 'esearch.fcgi?'
    search_term = '&term=' + query
    search_usehistory = '&usehistory=y'
    search_rettype = '&rettype=xml'
    search_date = '+2000[pdat]'
    search_retmax ='&retmax=100000000'
    search_mindate = '&mindate='+mindate
    search_maxdate = '&maxdate='+maxdate

    # Create search URL
    search_url = base_url \
        + search_eutil \
        + db+ search_term \
        + search_mindate \
        + search_maxdate \
        + search_usehistory \
        + search_retmax \
        + search_rettype

    print("Search URL", search_url)
    f = urllib.request.urlopen(search_url)
    search_data = f.read().decode('utf-8')

    # Fetch PMIDs and print in XML format and convert it to Dictionary
    search_data_xml = requests.get(search_url)
    dict_data = xmltodict.parse(search_data_xml.content)
    count = int(dict_data["eSearchResult"]["Count"])
    web = ((dict_data["eSearchResult"]["WebEnv"]))
    key = (dict_data["eSearchResult"]["QueryKey"])
    print("key is", key)
    print("Count value is ", count)

    ids = dict_data["eSearchResult"]["IdList"]["Id"]
    print(len(ids))
    
    # Write the results to a file
    writer = open('results-refined/{}.txt'.format(item), 'w')
    for the_id in ids:
        writer.write(str(the_id))
        writer.write("\n")
    writer.close()


In [ ]:
# save all ids for each related search
index = 0
for item in query_list:
    writer = open('{}.txt'.format(item), 'w')
    for id in final_ids[index]:
        writer.write(str(id))
        writer.write("\n")
    index += 1


In [3]:
# Aggreagte all pmids from all searches and save in a text file, and create a dataframe (excel) from all searches
cwd = os.getcwd()
path = "results-refined(corpora_2)" # use your path
#print(path)
#print("cwd is......",cwd)
#print(os.path.join(path, "*.csv"))
all_files = glob.glob(os.path.join(path, "*.txt"))
#print(all_files)
all_ids = []
dataframe = {}
for filename in all_files:
    fname = os.path.basename(filename).split(".txt")[0]
    #print(fname)
    if(fname == "all-ids"):
        continue
    df_file = open(filename,"r")
    temp = []
    for line in df_file:
        temp.append(line.split("\n")[0]) # to be added to dataframe of all searches
        all_ids.append(line.split("\n")[0])
        dataframe[fname] = temp

unique_ids = set(all_ids)
print(len(unique_ids))
f = open("results_refined_unique_id.txt","w" )
for item in unique_ids:
    f.write(item)
    f.write("\n")
print("all ids (not unique", len(all_ids))
df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in dataframe.items()]))
#df_dataframe = pd.DataFrame.from_dict(dataframe)
df.to_excel("all-searches_excel_try_2.xlsx",engine="xlsxwriter",index=False)
file = open("results-refined(corpora_2)/all-ids.txt","w")
for id in unique_ids:
    file.write(id)
    file.write("\n")
file.close()



19667
all ids (not unique 85095
